### Improving Epidemiological Diseases Mitigation: A case study of Monkeypox using Agent-based Modelling and Machine Learning Approaches.

#### STAGE 3: Continous cleaning, replacing missing values, feature engineering,Encode categorical variables into numeri, dataset train,test splitting, Models implementation.

#### At this stage I also eccounter difficulty of getting different result from RF, XGBoost, DT, and Neural Network. Resolve by setting RANDOM_STATE seed to 42.

In [3]:
#!pip install xgboost

In [102]:
# import required libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [103]:
# imports the Python warnings module to provides a mechanism for handling warnings that occur during program execution.
import warnings
warnings.filterwarnings('ignore')

In [104]:
# Load data
df_mkp = pd.read_csv('Monkeypox.csv')
df_mkp.head(5)


,ID,Status,Location,City,Country,Age,Gender,Date_onset,Date_confirmation,Symptoms,...,Travel_history_location,Travel_history_country,Genomics_Metadata,Confirmation_method,Source,Source_II,Date_entry,Date_last_modified,Source_III,Country_ISO3
0,1.0,confirmed,Guy's and St Thomas Hospital London,London,England,NaN,NaN,29/04/2022,06/05/2022,rash,...,Lagos and Delta States,Nigeria,West African Clade,RT-PCR,https://www.gov.uk/government/news/monkeypox-c...,https://www.who.int/emergencies/disease-outbre...,18/05/2022,18/05/2022,NaN,GBR
1,2.0,confirmed,Guy's and St Thomas Hospital London,London,England,NaN,NaN,05/05/2022,12/05/2022,rash,...,NaN,NaN,West African Clade,RT-PCR,https://www.gov.uk/government/news/monkeypox-c...,NaN,18/05/2022,18/05/2022,NaN,GBR
2,3.0,confirmed,London,London,England,NaN,NaN,30/04/2022,13/05/2022,vesicular rash,...,NaN,NaN,West African Clade,RT-PCR,https://www.gov.uk/government/news/monkeypox-c...,NaN,18/05/2022,18/05/2022,NaN,GBR
3,4.0,confirmed,London,London,England,NaN,male,NaN,15/05/2022,vesicular rash,...,NaN,NaN,West African Clade,NaN,https://www.gov.uk/government/news/monkeypox-c...,NaN,18/05/2022,18/05/2022,NaN,GBR
4,5.0,confirmed,London,London,England,NaN,male,NaN,15/05/2022,vesicular rash,...,NaN,NaN,West African Clade,NaN,https://www.gov.uk/government/news/monkeypox-c...,NaN,18/05/2022,18/05/2022,NaN,GBR


In [105]:
#Check dataset shape: number of rows and columns
df_mkp.shape

(1133, 31)

In [106]:
#check for missing value
df_mkp.isna().sum()

ID                            1
Status                        1
Location                    657
City                        762
Country                       1
Age                         965
Gender                      814
Date_onset                 1125
Date_confirmation           213
Symptoms                   1022
Hospitalised (Y/N/NA)      1001
Date_hospitalisation       1120
Isolated (Y/N/NA)           992
Date_isolation             1122
Outcome                    1132
Contact_comment            1097
Contact_ID                 1126
Contact_location           1129
Travel_history (Y/N/NA)    1004
Travel_history_entry       1121
Travel_history_start       1129
Travel_history_location    1106
Travel_history_country     1100
Genomics_Metadata          1110
Confirmation_method        1045
Source                        1
Source_II                   804
Date_entry                    2
Date_last_modified            2
Source_III                 1091
Country_ISO3                  0
dtype: i

## Data Preprocessing

In [107]:
# Input missing values based on the nature of the column

# For categorical columns, you can use mode imputation
categorical_columns = df_mkp.select_dtypes(include=['object']).columns
for column in categorical_columns:
    df_mkp[column].fillna(df_mkp[column].mode()[0], inplace=True)

# Display the number of missing values after handling
print("\nMissing values after handling:")
print(df_mkp.isnull().sum())


Missing values after handling:
ID                            1
Status                        0
Location                      0
City                          0
Country                       0
Age                           0
Gender                        0
Date_onset                    0
Date_confirmation             0
Symptoms                      0
Hospitalised (Y/N/NA)         0
Date_hospitalisation          0
Isolated (Y/N/NA)             0
Date_isolation                0
Outcome                       0
Contact_comment               0
Contact_ID                 1126
Contact_location              0
Travel_history (Y/N/NA)       0
Travel_history_entry          0
Travel_history_start          0
Travel_history_location       0
Travel_history_country        0
Genomics_Metadata             0
Confirmation_method           0
Source                        0
Source_II                     0
Date_entry                    0
Date_last_modified            0
Source_III                    0
Country_

In [108]:
#Check dataset shape: number of rows and columns
df_mkp.shape

(1133, 31)

In [109]:
# List of unwanted features/columns to be removed
unwanted_features = ['ID', 'Contact_ID', 'Source_II', 'Date_last_modified', 'Source_III', 'Source', 'Country_ISO3', 'Confirmation_method']


# Drop unwanted features from the DataFrame
df_mkp.drop(unwanted_features, axis=1, inplace=True)

# Display the DataFrame after removing unwanted features
df_mkp.head()

,Status,Location,City,Country,Age,Gender,Date_onset,Date_confirmation,Symptoms,Hospitalised (Y/N/NA),...,Outcome,Contact_comment,Contact_location,Travel_history (Y/N/NA),Travel_history_entry,Travel_history_start,Travel_history_location,Travel_history_country,Genomics_Metadata,Date_entry
0,confirmed,Guy's and St Thomas Hospital London,London,England,20-64,male,29/04/2022,06/05/2022,rash,Y,...,Recovered,Under investigation,Household,Y,04/05/2022,late April,Lagos and Delta States,Nigeria,West African Clade,18/05/2022
1,confirmed,Guy's and St Thomas Hospital London,London,England,20-64,male,05/05/2022,12/05/2022,rash,Y,...,Recovered,Index Case of household cluster,Household,N,16/05/2022,late April,Europe,Spain,West African Clade,18/05/2022
2,confirmed,London,London,England,20-64,male,30/04/2022,13/05/2022,vesicular rash,N,...,Recovered,Under investigation,Household,N,16/05/2022,late April,Europe,Spain,West African Clade,18/05/2022
3,confirmed,London,London,England,20-64,male,29/04/2022,15/05/2022,vesicular rash,Y,...,Recovered,Under investigation,Household,N,16/05/2022,late April,Europe,Spain,West African Clade,18/05/2022
4,confirmed,London,London,England,20-64,male,29/04/2022,15/05/2022,vesicular rash,Y,...,Recovered,Under investigation,Household,N,16/05/2022,late April,Europe,Spain,West African Clade,18/05/2022


In [110]:
# Drop NAN values
df_mkp.dropna(inplace=True)

In [15]:
# Specify the file path to save the CSV file
#file_path = 'Monkeypox_valuereplaced.csv'

# Save the cleaned DataFrame to a CSV file
#df_mkp.to_csv(file_path, index=False)

#print(f"Dataset has been saved to: {file_path}")

Dataset has been saved to: Monkeypox_valuereplaced.csv


In [111]:
## Cities in the Top Countries with the higher cases: Spain
df_mkp[df_mkp['Country'] == 'Spain'].groupby('City')['Status'].count()

City
03/06/2022         1
Almeria            3
Aragon             1
Barcelona          1
Cadiz              3
Costa del Sol      1
Fuerteventura      2
Gran Canaria      16
Madrid           275
Malaga            16
Seville            2
Tenerife           4
Valencia           2
Valladolid         1
Name: Status, dtype: int64

In [112]:
# An unidentify city name is detected '03/06/2022', needs to be replace with most occurence city. 
# Calculate the mode of the 'City' column
# Convert 'City' column to string type to handle potential variations
#df_mkp['City'] = df_mkp['City'].astype(str)

# Drop the rows where 'City' is '03/06/2022' as it's not relevant
df_mkp.drop(df_mkp[df_mkp['City'] == '03/06/2022'].index, inplace=True)

In [113]:
# Check to confirm removal of error data
df_mkp[df_mkp['Country'] == 'Spain'].groupby('City')['Status'].count()

City
Almeria            3
Aragon             1
Barcelona          1
Cadiz              3
Costa del Sol      1
Fuerteventura      2
Gran Canaria      16
Madrid           275
Malaga            16
Seville            2
Tenerife           4
Valencia           2
Valladolid         1
Name: Status, dtype: int64

In [114]:
#Check dataset shape: number of rows and columns
df_mkp.shape

(1132, 23)

In [115]:
# Encode categorical variables into numeric for better model impllementation
categorical_features = df_mkp.select_dtypes(include=["object"]).columns.tolist()
for feature in categorical_features:
    le = LabelEncoder()
    df_mkp[feature] = le.fit_transform(df_mkp[feature])

In [116]:
# Display dataset
df_mkp.head(10)

,Status,Location,City,Country,Age,Gender,Date_onset,Date_confirmation,Symptoms,Hospitalised (Y/N/NA),...,Outcome,Contact_comment,Contact_location,Travel_history (Y/N/NA),Travel_history_entry,Travel_history_start,Travel_history_location,Travel_history_country,Genomics_Metadata,Date_entry
0,0,47,42,12,3,2,5,4,23,1,...,0,12,0,1,0,2,6,10,1,5
1,0,47,42,12,3,2,0,5,23,1,...,0,8,0,0,3,2,5,13,1,5
2,0,59,42,12,3,2,6,6,31,0,...,0,12,0,0,3,2,5,13,1,5
3,0,59,42,12,3,2,5,7,31,1,...,0,12,0,0,3,2,5,13,1,5
4,0,59,42,12,3,2,5,7,31,1,...,0,12,0,0,3,2,5,13,1,5
5,0,59,42,12,3,2,5,7,31,0,...,0,12,0,0,3,2,5,13,1,5
6,0,75,54,12,3,2,5,7,31,1,...,0,12,0,1,3,2,8,5,1,5
7,0,55,39,36,2,2,5,8,28,0,...,0,12,0,0,3,2,5,13,1,5
8,0,55,39,36,2,2,5,8,28,0,...,0,12,0,0,3,2,5,13,1,5
9,0,55,39,36,2,2,5,8,28,0,...,0,12,0,0,3,2,5,13,1,5


In [117]:
# Check number of columns and rows
df_mkp.shape

(1132, 23)

In [85]:
# Specify the file path to save the CSV file
#file_path = 'Monkeypox_numericencode.csv'

# Save the cleaned DataFrame to a CSV file
#df_mkp.to_csv(file_path, index=False)

#print(f"Dataset has been saved to: {file_path}")

Dataset has been saved to: Monkeypox_numericencode.csv


In [118]:
# Display  first 50 dataset from backward
df_mkp.tail(50)

,Status,Location,City,Country,Age,Gender,Date_onset,Date_confirmation,Symptoms,Hospitalised (Y/N/NA),...,Outcome,Contact_comment,Contact_location,Travel_history (Y/N/NA),Travel_history_entry,Travel_history_start,Travel_history_location,Travel_history_country,Genomics_Metadata,Date_entry
1083,0,115,47,1,7,2,5,1,15,0,...,0,12,0,1,3,2,14,13,1,2
1084,2,80,18,4,1,2,5,1,17,0,...,0,12,0,1,3,2,5,13,1,2
1085,0,122,45,14,3,2,5,2,15,0,...,0,12,0,1,3,2,5,13,1,2
1086,0,122,45,14,3,2,5,2,15,0,...,0,12,0,1,3,2,5,13,1,2
1087,0,122,45,14,3,2,5,2,15,0,...,0,12,0,1,3,2,5,13,1,2
1088,0,122,45,14,3,2,5,2,15,0,...,0,12,0,1,3,2,5,13,1,2
1089,0,122,45,14,3,2,5,2,15,0,...,0,12,0,1,3,2,5,13,1,2
1090,0,122,45,14,3,2,5,2,15,0,...,0,12,0,1,3,2,5,13,1,2
1091,0,122,45,14,3,2,5,2,15,0,...,0,12,0,1,3,2,5,13,1,2
1092,0,122,45,14,3,2,5,2,15,0,...,0,12,0,1,3,2,5,13,1,2


In [119]:
# Count Status by category
df_mkp['Status'].value_counts()

0    920
1    145
2     67
Name: Status, dtype: int64

In [19]:
#!pip install --upgrade scikit-learn
#!pip install imbalanced-learn
#!pip install scikit-learn==0.24.2
#!pip install imbalanced-learn==0.8.0

In [121]:
# Import libraries
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

In [122]:
# Separate features and target variable
X = df_mkp.drop('Status', axis=1)
y = df_mkp['Status']

In [123]:
# Initialize SMOTE (Synthetic Minority Over-sampling Technique)
smote = SMOTE(random_state=42)

In [124]:
# Apply SMOTE to balance the dataset
X_resampled, y_resampled = smote.fit_resample(X, y)

In [125]:
# Combine the resampled features and target variable into a new DataFrame
df_resampled = pd.concat([pd.DataFrame(X_resampled, columns=X.columns), pd.Series(y_resampled, name='Status')], axis=1)

In [126]:
# Check the distribution after balancing
print(df_resampled['Status'].value_counts())

0    920
1    920
2    920
Name: Status, dtype: int64


In [127]:
# Split data
# Separate features and target variable
X = df_mkp.drop('Status', axis=1)
y = df_mkp['Status']

#convert y back to Dataframe
y=y.to_frame()
 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [128]:
y_train

,Status
12,0
381,1
717,0
497,0
964,0
...,...
121,0
1044,0
1096,0
860,0


In [129]:
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (905, 22)
X_test shape: (227, 22)
y_train shape: (905, 1)
y_test shape: (227, 1)


## Model Implementation and Evaluation

In [130]:
# Model Implementation and Evaluation
models = {}

In [131]:
# Support Vector Machine (SVM)
svm = SVC(kernel="linear", probability=True)  # Enable probability estimates
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)

# Use predict_proba for obtaining class probabilities
y_pred_svm_probs = svm.predict_proba(X_test)

svm_metrics = {
    "Accuracy": accuracy_score(y_test, y_pred_svm),
    "Precision": precision_score(y_test, y_pred_svm, average='weighted'),
    "Recall": recall_score(y_test, y_pred_svm, average='weighted'),
    "F1-score": f1_score(y_test, y_pred_svm, average='weighted'),
    "AUC-ROC": roc_auc_score(y_test, y_pred_svm_probs, multi_class='ovr'),
}

In [132]:
print(svm_metrics)

{'Accuracy': 0.8193832599118943, 'Precision': 0.8012606998316679, 'Recall': 0.8193832599118943, 'F1-score': 0.7902084563331668, 'AUC-ROC': 0.8952734629277441}


In [133]:

# Random Forest (RF)
# Define a random state to maintain same result at every run
RANDOM_STATE = 42

# Random Forest (RF)
rf = RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

# Store metrics directly
models["RF"] = {
    "metrics": {
        "Accuracy": accuracy_score(y_test, y_pred_rf),
        "Precision": precision_score(y_test, y_pred_rf, average='weighted'),
        "Recall": recall_score(y_test, y_pred_rf, average='weighted'),
        "F1-score": f1_score(y_test, y_pred_rf, average='weighted'),
        "AUC-ROC": roc_auc_score(y_test, rf.predict_proba(X_test), multi_class='ovr'),
    },
    "model": rf  # Optionally store the model itself 
}

# Output the stored metrics
print(models["RF"])

{'metrics': {'Accuracy': 0.960352422907489, 'Precision': 0.9630674409403954, 'Recall': 0.960352422907489, 'F1-score': 0.9612857492235397, 'AUC-ROC': 0.989767352154819}, 'model': RandomForestClassifier(random_state=42)}


In [135]:
# Decision Tree (DT)
# Add a random state for reproducibility
RANDOM_STATE = 42

# Decision Tree (DT)
dt = DecisionTreeClassifier(random_state=RANDOM_STATE)
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)

models["DecisionTree"] = {
    "metrics": {
        "Accuracy": accuracy_score(y_test, y_pred_dt),
        "Precision": precision_score(y_test, y_pred_dt, average='weighted'),
        "Recall": recall_score(y_test, y_pred_dt, average='weighted'),
        "F1-score": f1_score(y_test, y_pred_dt, average='weighted'),
        "AUC-ROC": roc_auc_score(y_test, dt.predict_proba(X_test), multi_class='ovr'),
    },
    "model": dt  # Optionally store the model itself 
}

# Print the metrics
print(models["DecisionTree"])

{'metrics': {'Accuracy': 0.9515418502202643, 'Precision': 0.9569500159968498, 'Recall': 0.9515418502202643, 'F1-score': 0.9536912935051284, 'AUC-ROC': 0.952449081180568}, 'model': DecisionTreeClassifier(random_state=42)}


In [137]:
# XGBoost
xgb_model = xgb.XGBClassifier()
models["XGBoost"] = xgb_model
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)
models["XGBoost"] = {
    "Accuracy": accuracy_score(y_test, y_pred_xgb),
    "Precision": precision_score(y_test, y_pred_xgb, average='weighted'),
    "Recall": recall_score(y_test, y_pred_xgb, average='weighted'),
    "F1-score": f1_score(y_test, y_pred_xgb, average='weighted'),
    "AUC-ROC": roc_auc_score(y_test, xgb_model.predict_proba(X_test), multi_class='ovr'),
}

print(models["XGBoost"])

{'Accuracy': 0.9427312775330396, 'Precision': 0.9538742251673064, 'Recall': 0.9427312775330396, 'F1-score': 0.9466552843436243, 'AUC-ROC': 0.990064870726659}


In [138]:
# import require library

import tensorflow as tf
import random
import os

# Set the random seeds
RANDOM_STATE = 42
os.environ['PYTHONHASHSEED']=str(RANDOM_STATE)
random.seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)
tf.random.set_seed(RANDOM_STATE)

# Neural Network (Multiclass)
model_nn = Sequential()
model_nn.add(Dense(128, activation="relu", input_shape=(X_train.shape[1],)))
model_nn.add(Dense(64, activation="relu"))
model_nn.add(Dense(len(df_mkp['Status'].unique()), activation="softmax"))  # Adjust the number of classes
model_nn.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# One-hot encode the target variable
y_train_one_hot = to_categorical(y_train)

# Train the model
model_nn.fit(X_train, y_train_one_hot, epochs=10, batch_size=32)

# Predictions
y_pred_nn = model_nn.predict(X_test)

# Metrics
metrics_nn = {
    "Accuracy": accuracy_score(y_test, np.argmax(y_pred_nn, axis=1)),
    "Precision": precision_score(y_test, np.argmax(y_pred_nn, axis=1), average='weighted'),
    "Recall": recall_score(y_test, np.argmax(y_pred_nn, axis=1), average='weighted'),
    "F1-score": f1_score(y_test, np.argmax(y_pred_nn, axis=1), average='weighted'),
    "AUC-ROC": roc_auc_score(y_test, y_pred_nn, multi_class='ovr'),
}

print("Neural Network Metrics:", metrics_nn)

Epoch 1/10
29/29 [==============================] - 2s 3ms/step - loss: 0.8154 - accuracy: 0.7558
Epoch 2/10
29/29 [==============================] - 0s 3ms/step - loss: 0.3928 - accuracy: 0.8343
Epoch 3/10
29/29 [==============================] - 0s 3ms/step - loss: 0.3199 - accuracy: 0.8541
Epoch 4/10
29/29 [==============================] - 0s 3ms/step - loss: 0.3254 - accuracy: 0.8486
Epoch 5/10
29/29 [==============================] - 0s 3ms/step - loss: 0.3306 - accuracy: 0.8586
Epoch 6/10
29/29 [==============================] - 0s 3ms/step - loss: 0.3038 - accuracy: 0.8597
Epoch 7/10
29/29 [==============================] - 0s 3ms/step - loss: 0.3020 - accuracy: 0.8729
Epoch 8/10
29/29 [==============================] - 0s 4ms/step - loss: 0.2742 - accuracy: 0.8785
Epoch 9/10
29/29 [==============================] - 0s 3ms/step - loss: 0.2849 - accuracy: 0.8575
Epoch 10/10
8/8 [==============================] - 0s 2ms/step
Neural Network Metrics: {'Accuracy': 0.8281938325991189